In [8]:
from bs4 import BeautifulSoup 
from urllib.request import Request, urlopen
import pandas as pd
import json
import time


# PARTIE 1 : COLLECTE DE DONNEES SUR SPRINGER

In [9]:
# Clé du moteur de recherche Springer 
# Remarque : un filtre est appliquée pour reprendre la discipline "Computer Science" et "Article"
KEY = '("Natural Language Processing") AND ("Multi-label")'

In [10]:
# Collecte le code source d'une page web d'un url donné en argument 
def collecte_source_code(url : str) -> str :
    try : 
        req = Request(url,headers={'User-Agent': 'Chrome/71.0.3578.98'})
        webpage = urlopen(req).read()
        source_code = BeautifulSoup(webpage,'html.parser') 
        return source_code
    except Exception as e :
        print('erreur de chargement de la page... opération recommencée') 
        collecte_source_code(url)

# Cette fonction compte le nombre de page de résultat de recherche pour la clé de recherche 
def page_counter(first_url : str) -> int :
    soup = collecte_source_code(first_url)
    number_page  = soup.find('span', { "class" : "number-of-pages"}).getText()
    return number_page  

# Cette fonctionne transforme la clé de recherche en liste de liens de chaque page de résultat
def transform_key_into_url(key : str) -> list :    
    key = key.replace(' ','+')
    key = key.replace('(','%28')
    key = key.replace(')','%29')
    key = key.replace('"','%27')
    url_list = ['https://rd.springer.com/search?query=' + key + '&facet-discipline="Computer+Science"&facet-content-type="Article"']
    number_page = int(page_counter(url_list[0]))
    for i in range(2, number_page, 1):
        i = str(i)
        url_list.append('https://rd.springer.com/search/page/' + i +'?query=%28%27Natural+Language+Processing%27%29+AND+%28%27Multi-label%27%29&facet-discipline=%22Computer+Science%22&facet-content-type=%22Article%22')
    return url_list

# collecte une liste de titre, date, type de publication, mots clé pour chaque article 
# et crée une dataframe avec ces données
def collect_data(article_list : list) :
    i = 1
    title_list = []
    date_list = []
    keywords_list = []
    publication_type_list = []
    for article in article_list : 
        print("chargement des données de l'article", i)
        soup = collecte_source_code(article)
        try :
            title = soup.find('meta', {'name':'dc.title'}).get('content') # title
            title_list.append(title)
        except Exception as e : 
            title=0
            title_list.append(title)
        try : 
            publication_date = soup.find('meta', {'name':'dc.date'}).get('content') # date
            date_list.append(publication_date)
        except Exception as e : 
            publication_date=0
            date_list.append(publication_date)
        try : 
            script = soup.find('script', attrs={'type':'text/javascript'}).text
            script = script.replace('window.dataLayer = ','')
            script = script.replace(';','')
            json1_data = json.loads(script)[0] # transformation du script json en dictionnaire 
            keywords = json1_data['Keywords'] # key words
            keywords_list.append(keywords)
        except Exception as e : 
            keywords=0
            keywords_list.append(keywords)
        try :     
            publication_type = soup.find('meta', {'name':'dc.type'}).get('content') # publication type
            publication_type_list.append(publication_type)
        except Exception as e : 
            publication_type = 0 
            publication_type_list.append(publication_type)     
        i=i+1
    create_dataframe(title_list, date_list, keywords_list, publication_type_list)

# Création d'une dataframe à partir des listes de titre, date, type de publication, mots clé pour chaque article
def create_dataframe(title_list : list, date_list : list, keywords_list : list, publication_type_list : list):
       # crée un dictionnaire à partir des listes
        dict = {'Title': title_list, 'Date': date_list , 'Author Keywords': keywords_list, 'Publication type' : publication_type_list}
        # utilisation du dictionnaraire pour créer une dataframe
        df = pd.DataFrame(dict)
        df.to_csv('data/dataSPRINGER.csv')

# collecte les liens des pages de recherche et des articles et en extrait les données 
def scrapping_springer(KEY : str) :
    url_list = transform_key_into_url(KEY)
    i = 0
    article_list = []
    for url in url_list :
        soup = collecte_source_code(url)
        alllink = soup.find_all(class_= 'title') # liens des articles
        i+=1
        print('collecte des liens des articles sur la page', i)
        for link in alllink :
            article_list.append('https://rd.springer.com/' + link['href']) # tout les liens des articles 
    print("le nombre total d'article est de", len(article_list))
    collect_data(article_list)


In [11]:
# lance le scraping sur springer
scrapping_springer(KEY)

collecte des liens des articles sur la page 1
collecte des liens des articles sur la page 2
collecte des liens des articles sur la page 3
collecte des liens des articles sur la page 4
collecte des liens des articles sur la page 5
collecte des liens des articles sur la page 6
collecte des liens des articles sur la page 7
collecte des liens des articles sur la page 8
collecte des liens des articles sur la page 9
collecte des liens des articles sur la page 10
collecte des liens des articles sur la page 11
collecte des liens des articles sur la page 12
collecte des liens des articles sur la page 13
collecte des liens des articles sur la page 14
collecte des liens des articles sur la page 15
collecte des liens des articles sur la page 16
collecte des liens des articles sur la page 17
collecte des liens des articles sur la page 18
collecte des liens des articles sur la page 19
collecte des liens des articles sur la page 20
collecte des liens des articles sur la page 21
collecte des liens des

chargement des données de l'article 175
chargement des données de l'article 176
chargement des données de l'article 177
chargement des données de l'article 178
chargement des données de l'article 179
chargement des données de l'article 180
chargement des données de l'article 181
chargement des données de l'article 182
chargement des données de l'article 183
chargement des données de l'article 184
chargement des données de l'article 185
chargement des données de l'article 186
chargement des données de l'article 187
chargement des données de l'article 188
chargement des données de l'article 189
chargement des données de l'article 190
chargement des données de l'article 191
chargement des données de l'article 192
chargement des données de l'article 193
chargement des données de l'article 194
chargement des données de l'article 195
chargement des données de l'article 196
chargement des données de l'article 197
chargement des données de l'article 198
chargement des données de l'article 199


chargement des données de l'article 380
chargement des données de l'article 381
chargement des données de l'article 382
chargement des données de l'article 383
chargement des données de l'article 384
chargement des données de l'article 385
chargement des données de l'article 386
chargement des données de l'article 387
chargement des données de l'article 388
chargement des données de l'article 389
chargement des données de l'article 390
chargement des données de l'article 391
chargement des données de l'article 392
chargement des données de l'article 393
chargement des données de l'article 394
chargement des données de l'article 395
chargement des données de l'article 396
chargement des données de l'article 397
chargement des données de l'article 398
chargement des données de l'article 399
chargement des données de l'article 400
chargement des données de l'article 401
chargement des données de l'article 402
chargement des données de l'article 403
chargement des données de l'article 404


# PARTIE 2 : GENERATION D'UN CSV COMPTANT LES OCCURENCES DE MOTS-CLE 

In [12]:
# télécharger le csv
df = pd.read_csv('dataSPRINGER.csv')
# crée une dataframe avec les "Author Keywords"
df_words = df[['Author Keywords']]

In [13]:
# sépare les valeurs séparées par "," en colonnes 
df_words = df_words['Author Keywords'].str.split(",", expand = True) 

In [14]:
# regroupe toutes les colonnes en une seule colonne
df_words = df_words.stack().reset_index()

In [15]:
# supprime les colonnes inutilisées 
df_words.drop(columns=['level_0'], inplace=True)
df_words.drop(columns=['level_1'], inplace=True)

In [16]:
# supprime les lignes sans valeur
df_words = df_words.dropna()
df_words = df_words[df_words!=0]

In [17]:
# transformer toutes les lettres en minuscule
df_words = df_words.applymap(lambda s:s.lower() if type(s) == str else s)

In [18]:
# supprimer les espaces à gauche et à droite des mots 
df_words[0] = (df_words[0]).str.lstrip()
df_words[0] = (df_words[0]).str.rstrip()

In [19]:
# compte le nombre d'occurence des keywords
df_words = df_words[0].value_counts()

In [20]:
# télécharge le csv avec le nombre d'occurence par mot
df_words.to_csv('key_words.csv')